In [1]:
import sys
import nltk
import sklearn
import pandas
import numpy

print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))

Python: 3.7.3 (default, Jun 24 2019, 04:54:02) 
[GCC 9.1.0]
NLTK: 3.4.1
Scikit-learn: 0.21.2
Pandas: 0.24.2
Numpy: 1.16.4


In [21]:
# Carregando dados
import pandas as pd
import numpy as np

df = pd.read_csv('data/dataset_tratado.txt', header=None, encoding='utf-8', sep='\t')
df.fillna('x', inplace=True)
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
0    793 non-null bool
1    793 non-null object
dtypes: bool(1), object(1)
memory usage: 7.0+ KB
None
       0                                                  1
0  False  pretensa estadual deduz apel extrem esbarr ved...
1  False  embarg insist mesm razo recurs apresent oposic...
2  False  acorda ora recorr nao neg vigenc direit federa...
3  False  tribunal justic assent compreensa part det leg...
4  False  segund jurisprudenc superior tribunal justic a...


In [22]:
# verificação das classes dos acórdãos
classes = df[0]
print(classes.value_counts())

False    525
True     268
Name: 0, dtype: int64


In [15]:
from sklearn.preprocessing import LabelEncoder

# convertendo classes em binário
encoder = LabelEncoder()
binary = encoder.fit_transform(classes)

print(binary[:10])

[0 0 0 0 0 0 1 1 0 0]


In [24]:
# criação da lista de ementas
ementas = df[1]
print(ementas[:10])

0    pretensa estadual deduz apel extrem esbarr ved...
1    embarg insist mesm razo recurs apresent oposic...
2    acorda ora recorr nao neg vigenc direit federa...
3    tribunal justic assent compreensa part det leg...
4    segund jurisprudenc superior tribunal justic a...
5    sobrest recurs especial ate julgament recurs e...
6    inadmissivel exigenc recolh reu prisa requisit...
7    dentr limit leg vez caracteriz reincidenc agra...
8    sobr prescrica aca repetica indebit tributari ...
9    tratas agrav instrument interpost contr decisa...
Name: 1, dtype: object


In [25]:
from nltk.tokenize import word_tokenize

# criação do BOW (bag of words)
all_words = []

for ementa in ementas:
    words = word_tokenize(ementa)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [26]:
# Exibição do número total de palavras e as 15 mais comuns
print('Número de palavras: {}'.format(len(all_words)))
print('Palavras mais comuns: {}'.format(all_words.most_common(15)))

Número de palavras: 4905
Palavras mais comuns: [('nao', 1587), ('recurs', 724), ('lei', 633), ('art', 631), ('especial', 601), ('embarg', 593), ('n', 584), ('agrav', 555), ('ser', 430), ('tribunal', 418), ('regimental', 363), ('sumul', 354), ('dev', 346), ('acorda', 328), ('cort', 321)]


In [27]:
# Uso das 1500 palavras mais comuns como características
word_features = list(all_words.keys())[:1500]

In [28]:
# Função determina quais das 1500 palavras características estão contidas nas ementas

def find_features(ementa):
    words = word_tokenize(ementa)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# Por exemplo
features = find_features(ementas[0])
for key, value in features.items():
    if value == True:
        print(key)

pretensa
estadual
deduz
apel
extrem
esbarr
vedaco
cont
sumul
stj
med
necessari
reexam
conjunt
faticoprobatori
aut
bem
clausul
contratu
deslind
controvers
total
inviavel
agrav
improv


In [29]:
# Aplicando à todas as ementas
ementas_zip = list(zip(ementas, binary))
zipped_list_1 = ementas_zip[:]
zipped_list_2 = list(ementas_zip) 

# embaralhando lista
seed = 1
np.random.seed = seed
np.random.shuffle(zipped_list_2)

# chamando a função find_features para cada ementa
featuresets = [(find_features(text), label) for (text, label) in ementas_zip]

In [30]:
# Separação dos conjuntos de característas em treinamento e teste usando sklean
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)

In [31]:
print(len(training))
print(len(testing))

594
199


In [32]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Modelos para treinar
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=1000, random_state=0),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, classifier in models:
    nltk_model = SklearnClassifier(classifier)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 64.321608040201
Decision Tree Accuracy: 78.39195979899498
Random Forest Accuracy: 81.90954773869346


/usr/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy: 77.88944723618091
SGD Classifier Accuracy: 80.40201005025126
Naive Bayes Accuracy: 72.8643216080402
SVM Linear Accuracy: 76.88442211055276


In [33]:
# Classificador por votos
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
zipped_models_1 = models[:]
zipped_models_2 = list(models)


nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = zipped_models_2, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 76.88442211055276


In [34]:
# criação de classe de label de predição para conjunto de teste

txt_features_2, labels_2 = list(zip(*testing))
txt_features_1, labels_1 = txt_features_2[:], labels_2[:]
txt_features, labels = list(txt_features_2), list(labels_2)

prediction = nltk_ensemble.classify_many(txt_features)

In [35]:
# relatório de classificação

print(classification_report(labels, prediction))
print(accuracy)

              precision    recall  f1-score   support

           0       0.82      0.88      0.84       136
           1       0.68      0.57      0.62        63

    accuracy                           0.78       199
   macro avg       0.75      0.72      0.73       199
weighted avg       0.77      0.78      0.77       199

76.88442211055276
